
# WEB SCRAPING - Get Stock Headlines - [Using: Selenium, Cron ]

##   This web scraping will show new headlines on stocks for last 24 hours 

### Table of content:
      1.Load and Install libraries
      2.get data step by step
      3.create schedule by Linux & Windows

### 1. Load and install libraries

In [1]:
#install libraries
'''
! pip install selenium
conda install -c conda-forge python-chromedriver-binary 
'''

'\n! pip install selenium\nconda install -c conda-forge python-chromedriver-binary \n'

In [2]:
#import libraries

from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep 
import requests
import urllib
import pandas as pd

# create webdriver object
#driver = webdriver.Chrome(ChromeDriverManager().install())


### 2. Get data step by step

### select stock

In [3]:
stock = 'tesla'

### Create connection with browser 

#### *Connect to google*

In [4]:
driver = webdriver.Chrome('/home/idan/chromedriver_linux64/chromedriver')
driver.get('https://www.google.com/')

<b> <font color='grey' size="4"> submit stock name to server</font></b> 

In [5]:
search_bar = driver.find_element_by_xpath('//input[@class= "gLFyf gsfi"]')
search_bar.send_keys(stock)
search_bar.submit()
sleep(5)

#### *Click on 'news' tag*

In [6]:
select_news = driver.find_element_by_xpath('//div[@id = "hdtb-msb-vis"]//div[@class = "hdtb-mitem hdtb-imb"]//following-sibling::div[@class = "hdtb-mitem hdtb-imb"]//a[@class= "q qs"]')
select_news.click()
sleep(2)

#### *click on 'Tools' button* 

In [7]:
tools_button = driver.find_element_by_xpath('//a[@id= "hdtb-tls"]')
tools_button.click()
sleep(2)

#### *select time search bar*

In [8]:
select_time_search_bar = driver.find_element_by_xpath('//div[@id = "hdtbMenus"]//div[@class ="hdtb-mn-hd" and @role = "button"]//following-sibling::div[@class ="hdtb-mn-hd" and @role = "button"]//following-sibling::div[@class ="hdtb-mn-hd" and @role = "button"]')
select_time_search_bar.click()
sleep(2)

#### *select last 24 hours*

In [9]:
last_day = driver.find_element_by_xpath('//ul[@class = "hdtbU hdtb-mn-o"]//li[@id= "qdr_d"]')
last_day.click()
sleep(2)

#### *Get current url*

In [10]:
driver.current_url

'https://www.google.com/search?q=tesla&tbm=nws&source=lnt&tbs=qdr:d&sa=X&ved=0ahUKEwjK6dq-3_TrAhUOGuwKHcM3D0kQpwUIJA&biw=781&bih=872&dpr=1'

#### *Show Headlines in first page*

In [11]:
articles = driver.find_elements_by_xpath('//div[@class= "dbsr"]//span[@dir="ltr"]')
for article in articles :
    print(article.text)

Car and Driver
Tesla Tells Us How It Keeps Beating Nearly Everyone in the Range Game
Torque News
Tesla 2025 Pricing Predictions Made Possible By Battery and Manufacturing Advances
Mashable
Enter this giveaway for a chance to win a Tesla Model 3
CNN
Canadian police charge Tesla driver who was allegedly doing 90 miles per hour while asleep
The Verge
Tesla owner in Canada charged with ‘sleeping’ while driving over 90 mph
Ars Technica
Cop arrests apparently sleeping Tesla driver going 93mph
Yahoo Finance
WHAT TO EXPECT AT TESLA’S UPCOMING BATTERY DAY
MarketWatch
3 things to know about Tesla’s ‘battery day’
Fox Business
Tesla notches victory in case against ex-worker accused of sharing company data
Electrek
Tesla Megapacks help alleviate UK’s hundreds of millions in grid balancing costs from the pandemic


#### *Get Headlines From All pages by loop*

In [12]:
%%time

class get_headlines :
    
      def __init__ (self,stock_name):
            self.stock = stock_name
            self.articles = []
            self.website = []
            self.page = []
            self.table = []
            
      def connect_to_server(self):
            self.driver = webdriver.Chrome('/home/idan/chromedriver_linux64/chromedriver')
            self.driver.get('https://www.google.com/')
            self.search_bar = self.driver.find_element_by_xpath('//input[@class= "gLFyf gsfi"]')
            self.search_bar.send_keys(self.stock)
            self.search_bar.submit()
            sleep(5)
            self.select_news = self.driver.find_element_by_xpath('//div[@id = "hdtb-msb-vis"]//div[@class = "hdtb-mitem hdtb-imb"]//following-sibling::div[@class = "hdtb-mitem hdtb-imb"]//a[@class= "q qs"]')
            self.select_news.click()
            sleep(2)
            self.tools_button = self.driver.find_element_by_xpath('//a[@id= "hdtb-tls"]')
            self.tools_button.click()
            sleep(2)
            self.select_time_search_bar = self.driver.find_element_by_xpath('//div[@id = "hdtbMenus"]//div[@class ="hdtb-mn-hd" and @role = "button"]//following-sibling::div[@class ="hdtb-mn-hd" and @role = "button"]//following-sibling::div[@class ="hdtb-mn-hd" and @role = "button"]')
            self.select_time_search_bar.click()
            sleep(2)
            self.last_day = self.driver.find_element_by_xpath('//ul[@class = "hdtbU hdtb-mn-o"]//li[@id= "qdr_d"]')
            self.last_day.click()
            sleep(2)

    
      def articles_per_page (self):  
           self.tag = self.driver.find_elements_by_xpath('//div[@class= "dbsr"]//span[@dir="ltr"]')
           i=0
           while i <= len(self.tag):      
                self.website.append(self.tag[i].text)
                i+=1
                self.articles.append(self.tag[i].text)
                i+=1
                if i >= len(self.tag) :
                    break
        
           self.table = pd.DataFrame(list(zip(self.website,self.articles)),columns= ['website','headline'])   
    
      
      def page_number (self):
         self.connect_to_server()
         self.pages_batch = self.driver.find_elements_by_tag_name('td')
         self.page_index = 1 
         self.last_page = int(self.pages_batch[-2].text)

         while self.page_index <= self.last_page :
              #print(f'\n\npage {self.page_index}')
              self.articles_per_page()
              
              self.page = self.driver.find_elements_by_xpath('//span[@dir ="rtl"]')
              self.page[-1].click()
              sleep(5)
              self.pages_batch = self.driver.find_elements_by_tag_name('td')
              self.last_page = int(self.pages_batch[-2].text)  
              self.page_index +=1 
              self.previous_page = int(self.pages_batch[-3].text)  
              #print(f'\\npage_index={self.page_index},previous_page={self.previous_page},last_page={self.last_page}')
              
              if self.last_page < self.previous_page :
                 self.driver.close()
                 print('\n\nFinished !')
                 break
         
         print('\n\nFinished :)')
         self.driver.close()
       
      def output (self) :
          return self.table 
            
            
                
stock = 'tesla'     
    
if __name__ == '__main__' :
   activate = get_headlines(stock)
   activate.page_number() 
   headlines = activate.output()    





Finished :)
CPU times: user 511 ms, sys: 16.2 ms, total: 527 ms
Wall time: 2min 11s


#### *Show Output*

In [15]:
pd.set_option('display.max_rows',None)
display(headlines)

,website,headline
0,Car and Driver,Tesla Tells Us How It Keeps Beating Nearly Eve...
1,Torque News,Tesla 2025 Pricing Predictions Made Possible B...
2,Mashable,Enter this giveaway for a chance to win a Tesl...
3,CNN,Canadian police charge Tesla driver who was al...
4,The Verge,Tesla owner in Canada charged with ‘sleeping’ ...
5,Ars Technica,Cop arrests apparently sleeping Tesla driver g...
6,Yahoo Finance,WHAT TO EXPECT AT TESLA’S UPCOMING BATTERY DAY
7,MarketWatch,3 things to know about Tesla’s ‘battery day’
8,Fox Business,Tesla notches victory in case against ex-worke...
9,Electrek,Tesla Megapacks help alleviate UK’s hundreds o...


### 3.1 create schedule by Cron

In [ ]:
1. open terminal
2. emacs (for emacs editor)
3. insert in text editor : * 0 * * * /usr/bin/python3 /home/idan/Desktop/stock_WebScraping.py
4. save by name task_scheduler 
5. crostab task_scheduler.txt

### 3.2 create schedule by Windows

In [ ]:
1. create batch(text) file by notepad as follow:
  "C:\Users\Home\all\python\directories\python.exe" "C:/Users/python/script/directories/stock_WebScraping.py"

2. save notepad file as follow:
    notepad_name.bat
3. scroll and select .txt

4. open task scheduler => Activation Tab => Script folder: 
   insert C:\Users\Home\notepad\directory\notepad_name.bat
   
5. define your schedule